In [12]:
%load_ext autoreload

In [30]:
import sys
import os
import glob
sys.path.append('../VDAO_Access')
from Annotation import Annotation

In [33]:
vdao_dir = '/home/rafael/VDAO'
annot_files = os.path.join(vdao_dir, '**','*.txt')
annot_files = glob.glob(annot_file, recursive=True)
for file in annot_files:
    annot = Annotation(annotationFilePath=file)
    print(annot.IsValid())
    print(annot.totalFrames)


TypeError: unsupported operand type(s) for +: 'NoneType' and 'int'